<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Caminho_M%C3%ADnimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook ilustra a solução de um problema de caminho mínimo formulado como um problema de otimização linear em redes.

Note que este não é o método computacionalmente mais eficiente para resolver problemas de caminho mínimo. No entanto, é um método interessante do ponto de vista teórico.

In [ ]:
!pip install pyomo
!apt-get install coinor-cbc

In [1]:
import pyomo.environ as pyo

In [8]:
##Dados do problema

modelo = pyo.ConcreteModel()

##Conjunto de nós
n = 9
modelo.V = list(range(1,n+1))

origem = 1
destino = n

##Custos em cada arco
modelo.custos_arcos = {(1,2):1, (1,3):2, (2,4):6, (2,5):12, (3,4):3, (3,6):4,
                       (4,5):4, (4,6):3, (4,7):15, (4,8):7, (5,7):7, (6,8):7,
                       (6,9):15,(7,9):3, (8,9):10}

##Conjunto de nós predecessores
modelo.V_pred = {1:[], 2:[1], 3:[1], 4:[2,3],
                 5:[2,4], 6:[3,4], 7:[4], 8:[4,6], 9:[6,7,8]}

##Conjunto de nós sucessores
modelo.V_suc = {1:[2,3],2:[4,5], 3:[4,6], 4:[5,6,7,8],
                5:[7], 6:[8,9], 7:[9], 8:[9], 9:[]}


##Conjunto de arcos (define a topologia da rede)
modelo.A = list(modelo.custos_arcos.keys())

##Demandas (perceba que a demanda só é diferente de zero nos nós de origem e destino)
modelo.b = {i:0 for i in modelo.V}
modelo.b[1] = 1
modelo.b[n] = -1



In [9]:
##Criação do modelo linear

##Variáveis de decisão
modelo.x = pyo.Var(modelo.A, domain=pyo.NonNegativeReals)

##Restrições
def res_conserv(modelo, i):

  expr =  sum(modelo.x[(i,k)] for k in modelo.V_suc[i]) - sum(modelo.x[(k,i)] for k in modelo.V_pred[i]) == modelo.b[i]

  return expr


modelo.res = pyo.Constraint(modelo.V, rule=res_conserv)

##Função-objetivo
modelo.obj = pyo.Objective(expr = sum(modelo.custos_arcos[(i,j)]*modelo.x[(i,j)] for (i,j) in modelo.A), sense=pyo.minimize)



In [10]:
solver = pyo.SolverFactory("cbc")
solver.solve(modelo, tee=True)


Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmppuw1_32w.pyomo.lp -stat=1 -solve -solu /tmp/tmppuw1_32w.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 0 (-10) rows, 0 (-16) columns and 0 (-30) elements
Statistics for presolved model


Problem has 0 rows, 0 columns (0 with objective) and 0 elements
There are 22011 singletons with no objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 0 (-10) rows, 0 (-16) columns and 0 (-30) elements
Empty

{'Problem': [{'Name': 'unknown', 'Lower bound': 21.0, 'Upper bound': 21.0, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 16, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.026128768920898438}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [16]:
for i in modelo.x.keys():
  print(modelo.x[i].name, pyo.value(modelo.x[i]))

print("\nComprimento mínimo = ", pyo.value(modelo.obj))

x[1,2] -0.0
x[1,3] 1.0
x[2,4] 0.0
x[2,5] 0.0
x[3,4] 0.0
x[3,6] 1.0
x[4,5] 0.0
x[4,6] 0.0
x[4,7] 0.0
x[4,8] 0.0
x[5,7] 0.0
x[6,8] 0.0
x[6,9] 1.0
x[7,9] 0.0
x[8,9] 0.0

Comprimento mínimo =  21.0
